In [1]:
import pandas as pd
from slugify import slugify
import json
import math

def formatINR(number):
    s, *d = str(number).partition(".")
    r = ",".join([s[x-2:x] for x in range(-3, -len(s), -2)][::-1] + [s[-3:]])
    return "".join([r] + d)

In [6]:
# read data

raw_data = pd.read_excel('/home/abhinav/sink/D4D/overview_data.xlsx', engine='openpyxl')
indicators = pd.read_excel('/home/abhinav/sink/D4D/Indicators.xlsx', engine='openpyxl')
dist_meta_data = {"morigaon":{"Title": "Morigaon", "About": "Morigaon district is an administrative district in the state of Assam in India. The district headquarters is located at Morigaon. The ancient place of occult Mayong is located in this district as well as Pobitora Wildlife Sanctuary.", "link": "https://morigaon.gov.in/"}, "nagaon":{"Title": "Nagaon", "About": "Nagaon is an administrative district in the Indian state of Assam. The undivided district was the most populous in the state as per 2011 census. The eastern, western, and southern fringes of the newly organized district were once ruled by different small feudal kings or their agents. It was a newly organized village system-hence called “Nagaon”, Na meaning new.", "link": "https://nagaon.assam.gov.in/"} }
dept_meta_data = {'panchayat-and-rural-development': {'Title': "Panchayat and Rural Development", 'Desc':'The Panchayat & Rural Development Department largely works to enhance livelihood opportunities, address chronic poverty, provides social security and works for economic inclusion of rural poor families. The Panchayat & Rural Development Department has been implementing a number of programmes in the rural area of the State by empowering rural poor through power of rights based law, aiming to transform the geography of poverty in the State.', 'schemes':['mgnrega']}, 
             'public-health-engineering': {'Title': "Public Health Engineering", 'Desc':'The Public Health Engineering Department has a mission to enhance quality of life of the people by ensuring sustainable safe drinking water and sanitation facilities and services along with promoting hygiene practices according to their choices and affordability.', 'schemes':['jjm']}}
scheme_meta_data = {"mgnrega": {"Name": "Mahatma Gandhi National Rural Employment Guarantee Act (MGNREGA)", "Title": "MGNREGA", "Desc": "The objective of MGNREGS is to enhance livelihood security of the rural masses. The scheme is meant to provide at least 100 days of wage employment per financial year (FY) to every rural household whose adult members are willing to do unskilled manual work. This is one of the world’s largest public sector employment programmes that provides guaranteed income through employment.", "dept_slug":"panchayat-and-rural-development", "logo": "/logo/mgnrega.webp"},
                    "jjm": {"Name": "Jal Jeevan Mission (JJM)", "Title": "JJM", "Desc": "The National Rural Drinking Water Programme / Jal Jeevan Mission is an initiative of the Government of India under the Ministry of Jal Shakti. It aims to provide safe and adequate drinking water to all households in rural India through provision of individual household tap connections by the year 2024.", "dept_slug":"public-health-engineering", "logo": "/logo/nhm.png"}}

In [7]:
#Merge indicator data into data file 
data = pd.merge(raw_data,indicators[['indicator_slug','indicator_name','indicator_type', 'indicator_selected', 'indicator_overview', 'indicator_explorer', 'indicator_dept', 'indicator_district', 'indicator_unit']],on='indicator_slug', how='left')


In [8]:
# filter data for overview page indicators and structure the column values.
# filter data for financial year latest

data = data[data.indicator_overview == 'yes']
data = data[data.financial_year == '2023-2024']

def get_dept_slug(x):
    
    for key in dept_meta_data:
        if x in dept_meta_data[key]['schemes']:
            return key
        
def get_dept_name(x):
    
    for key in dept_meta_data:
        if x in dept_meta_data[key]['schemes']:
            return dept_meta_data[key]["Title"]   
        
def get_type_title(x):
    
    if x == "district_profile":
        return "District Profile"
    elif x == "district_performance":
        return "District Performance"
    elif x == "target":
        return "Targets"
    else:
        return x

data['district_name'] = data['district_name'].map(lambda x: x.title())
data['district_slug'] = data['district_name'].map(lambda x: slugify(x))
data['scheme_slug'] = data['scheme_name'].map(lambda x: slugify(x))

data['dept_slug'] = data['scheme_slug'].map(lambda x: get_dept_slug(x))
data['dept_name'] = data['scheme_slug'].map(lambda x: get_dept_name(x))

data['indicator_type_title'] = data['indicator_type'].map(lambda x: get_type_title(x))  

# filter data for morigaon and nagaon district
data = data[data['district_name'].isin(["Morigaon", "Nagaon"])] 
    

data.head()

,district_name,district_code,financial_year,scheme_name,indicator_slug,indicator_report,indicator_value,date_of_update,indicator_name,indicator_type,...,indicator_overview,indicator_explorer,indicator_dept,indicator_district,indicator_unit,district_slug,scheme_slug,dept_slug,dept_name,indicator_type_title
6789,Morigaon,26,2023-2024,MGNREGA,awppd,Outlays & Outcomes,1665.53,2023-11-08,Average wage per personday,district_performance,...,yes,yes,yes,no,rupees,morigaon,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development,District Performance
6791,Nagaon,27,2023-2024,MGNREGA,awppd,Outlays & Outcomes,3093.39,2023-11-08,Average wage per personday,district_performance,...,yes,yes,yes,no,rupees,nagaon,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development,District Performance
6999,Morigaon,26,2023-2024,MGNREGA,cppdy,Outlays & Outcomes,2109.59,2023-11-08,Cost per personday,district_performance,...,yes,yes,yes,no,rupees,morigaon,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development,District Performance
7001,Nagaon,27,2023-2024,MGNREGA,cppdy,Outlays & Outcomes,3343.14,2023-11-08,Cost per personday,district_performance,...,yes,yes,yes,no,rupees,nagaon,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development,District Performance
10569,Morigaon,26,2023-2024,MGNREGA,tpdue,Financial Statement,19.29,2023-11-08,Total Payment Due,district_performance,...,yes,yes,yes,no,rupees lakhs,morigaon,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development,District Performance


In [9]:
# Overview Page Generator

base_json = {}

for index, row in data.iterrows():
    
    # insert district
    if  row.district_slug not in base_json:
        base_json[row.district_slug] = {
        "distTitle": row.district_name,
        "depts" :
        {
            row.dept_slug : 
            {"deptTitle": row.dept_name,
                "schemes" :
                {
                    row.scheme_slug : 
                    {
                        "schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                        "schemeDesc": scheme_meta_data[row.scheme_slug]['Desc'],
                        "lastUpdated": str(row.date_of_update),
                        "targetTitle": "Targets",
                        "targets": [] if row.indicator_type != "target" else [
                        {
                            "value": round(row.indicator_value*100, 2),
                            "label": row.indicator_name,
                            "description": row.indicator_name,
                            "type": "progress",
                            "min": 0,
                            "max": 100
                        }
                        ],
                        "profileTitle": "District Profile",
                        "profiles": [] if row.indicator_type != "district_profile" else [
                        {
                            "value": str(round(row.indicator_value, 2)) + " " + ("INR" if row.indicator_unit == "rupees" else "INR in Lakhs" if row.indicator_unit == "rupees lakhs" else 
                                                            "%" if row.indicator_unit == "percentage" else ""),
                            "label": row.indicator_name,
                            "description": row.indicator_name
                        }
                        ],
                        "performanceTitle": "District Performance",
                        "performances": [] if row.indicator_type != "district_performance" else [
                        {
                            "value": str(round(row.indicator_value, 2)) + " " + ("INR" if row.indicator_unit == "rupees" else "INR in Lakhs" if row.indicator_unit == "rupees lakhs" else 
                                                            "%" if row.indicator_unit == "percentage" else ""),
                            "label": row.indicator_name,
                            "description": row.indicator_name,
                            "type": "number"
                        }
                        ],
                    }
                }
            }
        }
    }


            
    elif  row.dept_slug not in base_json[row.district_slug]["depts"]:
        
        base_json[row.district_slug]["depts"][row.dept_slug] ={
                "deptTitle": row.dept_name,
                "schemes" :
                {
                    row.scheme_slug : 
                    {
                        "schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                        "schemeDesc": scheme_meta_data[row.scheme_slug]['Desc'],
                        "lastUpdated": str(row.date_of_update),
                        "targetTitle": "Targets",
                        "targets": [] if row.indicator_type != "target" else [
                        {
                            "value": round(row.indicator_value*100, 2),
                            "label": row.indicator_name,
                            "description": row.indicator_name,
                            "type": "progress",
                            "min": 0,
                            "max": 100
                        }
                        ],
                        "profileTitle": "District Profile",
                        "profiles": [] if row.indicator_type != "district_profile" else [
                        {
                            "value": str(round(row.indicator_value, 2)) + " " + ("INR" if row.indicator_unit == "rupees" else "INR in Lakhs" if row.indicator_unit == "rupees lakhs" else 
                                                            "%" if row.indicator_unit == "percentage" else ""),
                            "label": row.indicator_name,
                            "description": row.indicator_name
                        }
                        ],
                        "performanceTitle": "District Performance",
                        "performances": [] if row.indicator_type != "district_performance" else [
                        {
                            "value": str(round(row.indicator_value, 2)) + " " + ("INR" if row.indicator_unit == "rupees" else "INR in Lakhs" if row.indicator_unit == "rupees lakhs" else 
                                                            "%" if row.indicator_unit == "percentage" else ""),
                            "label": row.indicator_name,
                            "description": row.indicator_name,
                            "type": "number"
                        }
                        ],
                    }
                }
            }
        
    elif  row.scheme_slug not in base_json[row.district_slug]["depts"][row.dept_slug]["schemes"]:
        
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug] = {
                        "schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                        "schemeDesc": scheme_meta_data[row.scheme_slug]['Desc'],
                        "lastUpdated": str(row.date_of_update),
                        "targetTitle": "Targets",
                        "targets": [] if row.indicator_type != "target" else [
                        {
                            "value": round(row.indicator_value*100, 2),
                            "label": row.indicator_name,
                            "description": row.indicator_name,
                            "type": "progress",
                            "min": 0,
                            "max": 100
                        }
                        ],
                        "profileTitle": "District Profile",
                        "profiles": [] if row.indicator_type != "district_profile" else [
                        {
                            "value": str(round(row.indicator_value, 2)) + " " + ("INR" if row.indicator_unit == "rupees" else "INR in Lakhs" if row.indicator_unit == "rupees lakhs" else 
                                                            "%" if row.indicator_unit == "percentage" else ""),
                            "label": row.indicator_name,
                            "description": row.indicator_name
                        }
                        ],
                        "performanceTitle": "District Performance",
                        "performances": [] if row.indicator_type != "district_performance" else [
                        {
                            "value": str(round(row.indicator_value, 2)) + " " + ("INR" if row.indicator_unit == "rupees" else "INR in Lakhs" if row.indicator_unit == "rupees lakhs" else 
                                                            "%" if row.indicator_unit == "percentage" else ""),
                            "label": row.indicator_name,
                            "description": row.indicator_name,
                            "type": "number"
                        }
                        ],
                    }
    else:
        if row.indicator_type == "target" :
            base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["targets"].append(
                {"value": round(row.indicator_value*100, 2),"label": row.indicator_name,"description": row.indicator_name, "type": "progress", "min":0, "max":100}
                )
        elif  row.indicator_type == "district_profile":   
            base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["profiles"].append(
                {
                    "value": str(round(row.indicator_value, 2)) + " " + ("INR" if row.indicator_unit == "rupees" else "INR in Lakhs" if row.indicator_unit == "rupees lakhs" else 
                                                    "%" if row.indicator_unit == "percentage" else ""),
                    "label": row.indicator_name,
                    "description": row.indicator_name
                }
            )
        else:
            base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["performances"].append(
                {
                    "value": str(round(row.indicator_value, 2)) + " " + ("INR" if row.indicator_unit == "rupees" else "INR in Lakhs" if row.indicator_unit == "rupees lakhs" else 
                                                    "%" if row.indicator_unit == "percentage" else ""),
                    "label": row.indicator_name,
                    "description": row.indicator_name,
                    "type": "number"
                }
            )
            
        

        
with open('overview_data.json', 'w') as fp:
    json.dump(base_json, fp, indent=4)